In [1]:


#Import data manipulation libaries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



#### First data loading strategy

In [2]:
train_import = pd.read_csv("/kaggle/input/playground-series-s3e6/train.csv")
test_df = pd.read_csv("/kaggle/input/playground-series-s3e6/test.csv")
submission = pd.read_csv("/kaggle/input/playground-series-s3e6/sample_submission.csv")

#### Second data loading strategy
Loading another training data set
Adding a flag for the adversial cross-validation

In [3]:
test = pd.read_csv('/kaggle/input/playground-series-s3e6/test.csv')
test.drop("id",axis=1,inplace=True)
test['adv_val'] =  0

train = pd.read_csv('/kaggle/input/playground-series-s3e6/train.csv')
train.drop("id",axis=1,inplace=True)
train['adv_val'] =  1

original = pd.read_csv('/kaggle/input/paris-housing-price-prediction/ParisHousing.csv')
original['adv_val'] =  2

sample = pd.read_csv('/kaggle/input/playground-series-s3e6/sample_submission.csv')

#### Feature Engineering

In [4]:
#Define the target label

target='price'

#Create a dict - Why?
sets={'train':train,'test':test,'original':original}

In [5]:
#Function to drop rows that are duplicated in the train set and the original deck

def dropping_duplicates(data):
    data.drop_duplicates(inplace = True)
    
for n in {'train':train,'original':original}:
    print(f"\033[0;33;40m A number of duplicated rows in {n} is {sets[n].duplicated().sum()}, they were dropped \033[0;30;0m")
    dropping_duplicates(sets[n])

 A number of duplicated rows in train is 0, they were dropped 
 A number of duplicated rows in original is 0, they were dropped 


In [6]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

In [7]:
########################
### General Settings ###
########################

gpu_switch = 'ON'

###############################
### RepeatedStratifiedKFold ###
###############################

n_splits = 5
n_repeats =10
sm=SMOTE(sampling_strategy='minority')
#cv = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats = n_repeats, random_state=2023)
cv = RepeatedKFold(n_splits=n_splits, n_repeats = n_repeats, random_state=2023)
#cv = GroupKFold(n_splits=n_splits)

########################
### Define Weights   ###
########################

# Understand where and why the weights are used
weights = {0: 0.5009553158705701, 1: 262.19354838709677}

#### Define lists for feature engineering

In [8]:
num_cols = ['squareMeters','numberOfRooms','floors','cityPartRange','numPrevOwners',
            'made','basement','attic','garage','hasGuestRoom','cityCode'
            ]
cat_cols = ['hasYard','hasPool','isNewBuilt','hasStormProtector','hasStorageRoom',]
drop_col = []

#### Drop list from the test, train and original data set

In [9]:
#Drop the features in the list that have been define above
train.drop(drop_col,axis=1,inplace=True)
test.drop(drop_col,axis=1,inplace=True)
original.drop(drop_col,axis=1,inplace=True)

In [10]:
#Version 1
num_imp = SimpleImputer(strategy='mean')
cat_imp = SimpleImputer(strategy='most_frequent')
ohe = OneHotEncoder(handle_unknown='ignore',sparse = False,drop="first")

train[num_cols] = pd.DataFrame(num_imp.fit_transform(train[num_cols]),columns=num_cols)
train[cat_cols] = pd.DataFrame(cat_imp.fit_transform(train[cat_cols]),columns=cat_cols)
train_temp=pd.DataFrame(ohe.fit_transform(train[cat_cols]),columns=ohe.get_feature_names_out())
train=pd.concat([train.drop(cat_cols,axis=1),train_temp],axis=1) 

original[num_cols] = pd.DataFrame(num_imp.transform(original[num_cols]),columns=num_cols)
original[cat_cols] = pd.DataFrame(cat_imp.transform(original[cat_cols]),columns=cat_cols)
original_temp=pd.DataFrame(ohe.transform(original[cat_cols]),columns=ohe.get_feature_names_out())
original=pd.concat([original.drop(cat_cols,axis=1),original_temp],axis=1)    

test[num_cols] = pd.DataFrame(num_imp.transform(test[num_cols]),columns=num_cols)
test[cat_cols] = pd.DataFrame(cat_imp.transform(test[cat_cols]),columns=cat_cols)
test_temp=pd.DataFrame(ohe.transform(test[cat_cols]),columns=ohe.get_feature_names_out())
test=pd.concat([test.drop(cat_cols,axis=1),test_temp],axis=1) 

In [11]:
#Add improved version with Sklearn pipeline

In [12]:
#Combine train and priginbal data after the adversarial cross-validation
train=pd.concat([train,original],ignore_index=True)

In [13]:
#Drop the adversarial column
train=train.drop('adv_val',axis=1)
test=test.drop('adv_val',axis=1)
original=original.drop('adv_val',axis=1)

#### Define the features and target labels

In [14]:
# Drop the target from the dataframe to create the features
X = train.drop([target],axis=1)
#Create a series with the targets
y = train[target]
#Shuffle the data
X,y=shuffle(X,y,random_state=2023)
#Reset the index for the feature dataframe and the target labels
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

In [15]:
###########################
### additional dropping ###
###########################

#Drop outliers by row/sample/id
indexies =X[X['squareMeters']>99999].index
X=X.drop(indexies).reset_index(drop=True) #square meters
y=y.drop(indexies).reset_index(drop=True) #square meters

In [16]:
X['floors']=X['floors'].apply(lambda x: x if x<1000 else train['floors'].mean())
X['made']=X['made'].apply(lambda x: x if x<2022 else train['made'].mean() )
X['basement']=X['basement'].apply(lambda x: x if x<=10000 else train['basement'].mean() )
test['basement']=test['basement'].apply(lambda x: x if x<=10000 else train['basement'].mean() )
X['attic']=X['attic'].apply(lambda x: x if x<=10000 else train['attic'].mean() )
test['attic']=test['attic'].apply(lambda x: x if x<=10000 else train['attic'].mean() )
X['garage']=X['garage'].apply(lambda x: x if x<=1000 else train['garage'].mean() )
X['cityCode']=X['cityCode'].apply(lambda x: x if x<=100000 else int(train['cityCode'].mean()) )
test['cityCode']=test['cityCode'].apply(lambda x: x if x<=100000 else int(train['cityCode'].mean()) )
X['points']=X['hasYard_1']+X['hasPool_1']+X['isNewBuilt_1']+X['hasStormProtector_1']+X['hasStorageRoom_1']
X.drop(['hasYard_1','hasPool_1','isNewBuilt_1','hasStormProtector_1','hasStorageRoom_1'],axis=1,inplace=True)
test['points']=test['hasYard_1']+test['hasPool_1']+test['isNewBuilt_1']+test['hasStormProtector_1']+test['hasStorageRoom_1']
test.drop(['hasYard_1','hasPool_1','isNewBuilt_1','hasStormProtector_1','hasStorageRoom_1'],axis=1,inplace=True)

In [17]:
X=X.reset_index(drop=True)
y=y.reset_index(drop=True)

In [18]:
def StaSca_transform(X,test):
    StaSca = StandardScaler()
    X[num_cols] = pd.DataFrame(data = StaSca.fit_transform(X[num_cols]),columns = X[num_cols].columns)
    test[num_cols] = pd.DataFrame(data = StaSca.transform(test[num_cols]),columns = test[num_cols].columns)

In [19]:
def Box_transform(X,test):
    box_cols = ['fixed acidity','volatile acidity','residual sugar',
                 'chlorides','free sulfur dioxide','total sulfur dioxide',
                 'sulphates','alcohol']

    for column in box_cols: 
        X_temp,fitted_lambda = stats.boxcox(X[column]) 
        X[column]=X_temp 
        test_temp = stats.boxcox(test[column],fitted_lambda) 
        test[column]=test_temp

In [20]:
def perm_imp(model,data,target):
    X = data.to_numpy().copy()
    y = target.to_numpy().copy()
    permute = PermutationImportance(model,random_state=2023,n_iter =2,cv=10,scoring='neg_root_mean_squared_error').fit(X, y)
    eli5.show_weights(permute, feature_names = data.columns.tolist(),top=50)
    values = dict(zip(list(data.columns),list(permute.feature_importances_)))
    sorted_dict = {}
    sorted_keys = sorted(values, key=values.get)
    for w in sorted_keys:
        sorted_dict[w] = np.round(values[w],3)
    return sorted_dict

# Modelling

In [21]:
#Load the necessary packages
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
import base64
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random
import gc

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from mlxtend.preprocessing import minmax_scaling
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from scipy import stats
import optuna
import xgboost as xgb
from sklearn.ensemble import (RandomForestClassifier, RandomForestRegressor,AdaBoostClassifier, GradientBoostingClassifier, 
                              ExtraTreesClassifier, VotingClassifier,ExtraTreesRegressor,AdaBoostRegressor,GradientBoostingRegressor)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.utils import class_weight
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from sklearn.svm import SVR
from sklearn import datasets, linear_model
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.signal import argrelmin
from scipy.stats import skew
from scipy import stats


pd.set_option('display.max_columns', None)   

from sklearn.model_selection import train_test_split

import eli5
from eli5.sklearn import PermutationImportance
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,BatchNormalization,Dropout
from tensorflow.keras import utils
import tensorflow_addons as tfa
import keras_tuner
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from kerastuner import HyperParameters, Objective

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:78: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.


In [22]:
########################
### Define Weights   ###
########################


classes = y.unique()
weight = class_weight.compute_class_weight(class_weight='balanced', classes=classes, y=y)
weights = dict(zip(classes, list(weight)))

#######################
### Optuna Settings ###
#######################

optuna_study = True 

optuna_models={'LGBM' : True, 
               'XGB'  : False,
               'CAT'  : False,
               'AB'   : False,
               'GB'   : False,
               'ET'   : False,
               'RF'   : False,
               'LCV'  : False,
               'LR'   : False,
               'KNC'  : False,
               'SVC'  : False,
               'KERAS': False}

#######################
### Finish Settings ###
#######################
finish_set = True

finish_models={'LGBM' : False,
               'XGB'  : True,
               'CAT'  : True,
               'AB'   : False,
               'GB'   : False,
               'ET'   : False,
               'RF'   : True,
               'LCV'  : False,
               'LR'   : False,
               'KNC'  : False,
               'SVC'  : False,
               'KERAS': False}

#####################
### Consolidation ###
#####################
models={} # Dictionary to store the model
preds_val={} # Dictionary to store the predictions on the valdation data set
preds_test={} # Dictionary to store the predictions on the valdation test set

#######################
### X for modelling ###
#######################
X_model=X.copy() # This code also define a new variable before the modelling
test_model = test.copy()
y_model = y.copy()

In [23]:
################
### Settings ###
################
model_name='LGBM'
metric = 'rmse'

In [24]:
X = X_model
test = test_model
y=y_model

In [25]:
#Define a list of features which should be dropped before fitting with LGBM
LGBM_drop_list=[]

In [26]:
# Apply the drop to the dataframe
X = X_model.drop(LGBM_drop_list,axis=1)
test = test_model.drop(LGBM_drop_list,axis=1)
y=y_model

In [27]:

# code to decide if a GPU should be used or not
if gpu_switch == "ON":
    method = "gpu"
else:
    method = "cpu"

def objective(trial):
    param = {
     'device': method,# The logic to use GPU or CPU is applied here
     "metric":trial.suggest_categorical("metric", [metric]), 
     'verbosity': -1,
     'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0),
     'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0),
     'learning_rate': trial.suggest_float('learning_rate', 0.001,0.1),
     'num_leaves': trial.suggest_int('num_leaves', 2, 512),
     'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
     'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
     'early_stopping_round' : trial.suggest_int('early_stopping_round', 300, 300),
     'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
     'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
     'n_estimators' : trial.suggest_int('n_estimators', 100, 5000),
     "subsample":trial.suggest_categorical("subsample", [None]),
     "subsample_freq":trial.suggest_categorical("subsample_freq", [None]),
     "reg_alpha":trial.suggest_categorical("reg_alpha", [None]),
     "colsample_bytree":trial.suggest_categorical("colsample_bytree", [None]),
     "reg_lambda":trial.suggest_categorical("reg_lambda", [None]),
     #'class_weight':weights
             }
    
    results=[] # Create an empty list for the results
    n_iterations=[] # Create an empty list for the iterations
    for train_index, test_index in cv.split(X, y):
        train_X, valid_X = X.iloc[train_index], X.iloc[test_index]
        train_y, valid_y = y.iloc[train_index], y.loc[test_index]
        model = lgb.LGBMRegressor(**param).fit(train_X,train_y,
                                            eval_set=[(valid_X,valid_y)],
                                            callbacks=[lgb.log_evaluation(period=0, show_stdv=False)]
                                             )  
        result = np.sqrt(mean_squared_error(valid_y,model.predict(valid_X)))
        results.append(result)
    n=sum(results)/len(results)   
    return n

if  optuna_study == optuna_models[model_name]:
    study = optuna.create_study(pruner=optuna.pruners.HyperbandPruner(),
                                direction='minimize')
    study.optimize(objective, n_trials=10)
    print('Best trial:', study.best_trial.params)

[I 2023-03-30 20:04:06,052] A new study created in memory with name: no-name-a7013958-035d-4763-92cb-f7d208caa375
[I 2023-03-30 20:51:56,295] Trial 0 finished with value: 166660.1868473584 and parameters: {'metric': 'rmse', 'lambda_l1': 3.060719660119112, 'lambda_l2': 7.047499974981712, 'learning_rate': 0.02308290856029349, 'num_leaves': 79, 'feature_fraction': 0.4049630057822259, 'bagging_fraction': 0.691577664583863, 'early_stopping_round': 300, 'bagging_freq': 2, 'min_child_samples': 45, 'n_estimators': 4548, 'subsample': None, 'subsample_freq': None, 'reg_alpha': None, 'colsample_bytree': None, 'reg_lambda': None}. Best is trial 0 with value: 166660.1868473584.
[I 2023-03-30 21:07:51,247] Trial 1 finished with value: 161583.30124633355 and parameters: {'metric': 'rmse', 'lambda_l1': 6.15334004342203, 'lambda_l2': 1.9082673298280957, 'learning_rate': 0.0903097633070593, 'num_leaves': 143, 'feature_fraction': 0.748723631109214, 'bagging_fraction': 0.6767126534990171, 'early_stopping_

Best trial: {'metric': 'rmse', 'lambda_l1': 4.66704966709607, 'lambda_l2': 1.8000367135508502, 'learning_rate': 0.07451190761720193, 'num_leaves': 53, 'feature_fraction': 0.9173661149312436, 'bagging_fraction': 0.4886061270219261, 'early_stopping_round': 300, 'bagging_freq': 7, 'min_child_samples': 8, 'n_estimators': 2603, 'subsample': None, 'subsample_freq': None, 'reg_alpha': None, 'colsample_bytree': None, 'reg_lambda': None}


In [28]:
params_best = study.best_trial.params

In [29]:
if finish_models[model_name] and finish_set:
    results=[]
    for i,(train_index, test_index) in enumerate(cv.split(X, y)):
        j=i//n_splits
        start_time = time.time()
        train_X, valid_X = X.iloc[train_index], X.iloc[test_index]
        train_y, valid_y = y.iloc[train_index], y.iloc[test_index]
        model = lgb.LGBMRegressor(**params_best).fit(train_X,train_y,
                                                eval_set=[(valid_X,valid_y)],
                                                callbacks=[lgb.log_evaluation(period=0, show_stdv=False)])  
        
        models[f'{model_name}_{i}_{j}']=(model)
        preds_val[f'{model_name}_{i}_{j}']=(model.predict(valid_X))
        preds_test[f'{model_name}_{i}_{j}']=(model.predict(test))
        
        result = np.sqrt(mean_squared_error(valid_y,model.predict(valid_X)))
        results.append(result)
        print (f'\033[0;33;40m Step #{i}.' + f"--- {time.time() - start_time}s sec ---" + f"Auc result = {result} \033[0;30;0m")
    print (f'\033[0;35;40m Final LGB Result = {sum(results)/len(results)} \033[0;30;0m')